## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-12-20-52-00 +0000,wsj,Government reports on inflation and the labor ...,https://www.wsj.com/politics/policy/white-hous...
1,2025-11-12-20-50-56 +0000,nypost,Marty McHigh! NJ driver crashes his 1980s Delo...,https://nypost.com/2025/11/12/us-news/marty-mc...
2,2025-11-12-20-50-11 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/11/12/us/eps...
3,2025-11-12-20-49-45 +0000,nypost,Jeffrey Epstein fretted with NYT reporter abou...,https://nypost.com/2025/11/12/us-news/jeffrey-...
4,2025-11-12-20-48-40 +0000,wapo,House to vote on reopening government after 7-...,https://www.washingtonpost.com/business/2025/1...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2313/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
25,trump,40
75,shutdown,22
45,new,18
0,government,17
21,epstein,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
182,2025-11-12-13-56-28 +0000,nypost,Furious White House reveals Jeffrey Epstein vi...,https://nypost.com/2025/11/12/us-news/jeffrey-...,121
352,2025-11-11-22-13-00 +0000,wsj,Senate Minority Leader Chuck Schumer wasn’t on...,https://www.wsj.com/politics/policy/progressiv...,108
54,2025-11-12-19-28-42 +0000,nypost,Michael Wolff told Epstein he should try to en...,https://nypost.com/2025/11/12/us-news/michael-...,98
306,2025-11-12-02-00-00 +0000,wsj,As data went dark from the government shutdown...,https://www.wsj.com/economy/central-banking/th...,94
31,2025-11-12-20-03-02 +0000,startribune,Epstein email says Trump 'knew about the girls...,https://www.startribune.com/epstein-emails-rel...,91


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
182,121,2025-11-12-13-56-28 +0000,nypost,Furious White House reveals Jeffrey Epstein vi...,https://nypost.com/2025/11/12/us-news/jeffrey-...
306,94,2025-11-12-02-00-00 +0000,wsj,As data went dark from the government shutdown...,https://www.wsj.com/economy/central-banking/th...
27,58,2025-11-12-20-09-00 +0000,wsj,Atlanta Fed President Raphael Bostic will reti...,https://www.wsj.com/economy/central-banking/at...
185,55,2025-11-12-13-51-00 +0000,nyt,India’s Prime Minister Vows Justice After New ...,https://www.nytimes.com/2025/11/11/world/asia/...
273,46,2025-11-12-07-18-55 +0000,nypost,Marriott guests left ‘homeless’ as rental comp...,https://nypost.com/2025/11/12/business/marriot...
289,41,2025-11-12-03-32-40 +0000,nypost,Bank of America CEO Brian Moynihan seeks ‘impo...,https://nypost.com/2025/11/11/business/bank-of...
352,41,2025-11-11-22-13-00 +0000,wsj,Senate Minority Leader Chuck Schumer wasn’t on...,https://www.wsj.com/politics/policy/progressiv...
122,39,2025-11-12-16-50-12 +0000,nypost,Why Jack Schlossberg can’t count on Kennedy fa...,https://nypost.com/2025/11/12/us-news/why-jack...
71,37,2025-11-12-18-48-19 +0000,wapo,Turkish military plane falls from sky over Geo...,https://www.washingtonpost.com/world/2025/11/1...
270,35,2025-11-12-07-26-25 +0000,nypost,Chess federation files complaint against ex-wo...,https://nypost.com/2025/11/12/world-news/chess...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
